In [1]:
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import h5py
import nbimporter
from Functions import process_file, process_file_optimized

In [17]:
labs = pd.read_csv('D:/Labs_processed.csv')
labs.CHARTTIME = pd.to_datetime(labs.CHARTTIME)



# # Replace the ?'Cholesterol Ratio (Total-HDL)' with 'Cholesterol Ratio (Total:HDL)'
# labs.ITEMID = labs.ITEMID.replace('Cholesterol Ratio (Total-HDL)', 'Cholesterol Ratio (Total-HDL)')

# # Resave the file to the same path it was loaded from


# labs = labs.drop(columns=['Unnamed: 0'])
# labs.to_csv('D:/Labs_processed.csv', index=False)
labs.head()

,ROW_ID,SUBJECT_ID,ITEMID,CHARTTIME,VALUE,FLAG
0,281,p000003,pH,2101-10-12 16:07:00,7.39,0
1,282,p000003,SPECIMEN TYPE,2101-10-12 18:17:00,ART,0
2,283,p000003,Base Excess,2101-10-12 18:17:00,-1,0
3,284,p000003,Calculated Total CO2,2101-10-12 18:17:00,22,0
4,285,p000003,Free Calcium,2101-10-12 18:17:00,0.93,1


In [15]:
labs.ITEMID.unique().tolist()

['pH',
 'SPECIMEN TYPE',
 'Base Excess',
 'Calculated Total CO2',
 'Free Calcium',
 'Intubated',
 'Lactate',
 'pCO2',
 'pO2',
 'Temperature',
 'Anion Gap',
 'Bicarbonate',
 'Calcium, Total',
 'Chloride',
 'Creatinine',
 'Glucose',
 'Magnesium',
 'Phosphate',
 'Potassium',
 'Sodium',
 'Urea Nitrogen',
 'Vancomycin',
 'Hematocrit',
 'Alanine Aminotransferase (ALT)',
 'Asparate Aminotransferase (AST)',
 'Creatine Kinase (CK)',
 'Anisocytosis',
 'Basophils',
 'Eosinophils',
 'Hemoglobin',
 'Hypochromia',
 'Lymphocytes',
 'Macrocytes',
 'MCH',
 'MCHC',
 'MCV',
 'Microcytes',
 'Monocytes',
 'Neutrophils',
 'Platelet Count',
 'Poikilocytosis',
 'RDW',
 'Red Blood Cell Fragments',
 'Red Blood Cells',
 'White Blood Cells',
 nan,
 'Creatine Kinase, MB Isoenzyme',
 'Troponin I',
 'Alkaline Phosphatase',
 'Bilirubin, Total',
 'PTT',
 'Oxygen',
 'Oxygen Saturation',
 'PEEP',
 'Tidal Volume',
 'Ventilation Rate',
 'Ventilator',
 'Hematocrit, Calculated',
 'INR(PT)',
 'PT',
 'Albumin',
 'Amylase',
 '

In [10]:
#PRODUCE DATA FOR ALL TESTS
tests = [x.split('_')[1].split('.')[0] for x in os.listdir('D:/simedy/models/')]

#Or specify the tests you want to produce data for

# tests = [

# 'White Blood Cells', 'WBC Count'
# #CBC
# 'White Blood Cells', 'Red Blood Cells', 'Hemoglobin 2', 'Hematocrit',
# 'MCV', 'MCH', 'MCHC', 'RDW', 'Platelet Count',

# #CMP + Electrolytes
# 'Blood Glucose 2', 'Urea Nitrogen', 'Creatinine', 'Potassium', 'Sodium', 'Calcium, Total', 'Chloride', 'Free Calcium', 'Magnesium',
# 'Albumin', 'Asparate Aminotransferase (AST)', 'Bicarbonate',


# # Thyroid
# 'Thyroid Stimulating Hormone', 'Thyroxine (T4)', 'Triiodothyronine (T3)',
# 'Thyroxine (T4), Free', 'Calculated Thyroxine (T4) Index',

# Lipid
# 'Triglycerides', 'Cholesterol, Total', 'Cholesterol, LDL, Calculated',
# 'Cholesterol, HDL',

# 'Hematocrit', 'Potassium', 'Sodium', 'Creatinine', 'Chloride',
#  'Urea Nitrogen', 'Bicarbonate', 'Anion Gap',
#  'Platelet Count', 'Blood Glucose 2', 'Hemoglobin 2',
#  'MCHC', 'RDW', 'MCH', 'Red Blood Cells',
#  'MCV', 'White Blood Cells', 'Magnesium',
#  'Calcium, Total', 'Phosphate', 'pH', 'Calculated Total CO2',
#  'pO2', 'pCO2', 'PTT', 'INR(PT)', 'PT',
#  'Free Calcium', 'Bilirubin, Total', 'Alanine Aminotransferase (ALT)',
#  'Asparate Aminotransferase (AST)', 'Alkaline Phosphatase',  
#  'Lactate', 'Neutrophils', 'Lymphocytes', 'Monocytes', 'Eosinophils', 'Basophils', 
#  'Albumin', 'Creatine Kinase (CK)', 'Creatine Kinase, MB Isoenzyme',
#  'Lactate Dehydrogenase (LD)', 'Troponin T',
#  'Lipase', 'Amylase', 'Hypochromia',
#  'Anisocytosis', 'Macrocytes', 'Atypical Lymphocytes',
#  'Metamyelocytes', 'Myelocytes', 'Microcytes', 'Poikilocytosis',
#  'Polychromasia', 'Bilirubin, Direct', 'Bilirubin, Indirect', 'Fibrinogen, Functional',
#  'Thyroid Stimulating Hormone', 'Triglycerides', 'Granulocyte Count', 'CK-MB Index',
#  'Cholesterol, Total', 'Osmolality, Measured', 'Uric Acid', 'Cholesterol, HDL',
#  'Iron', 'Cholesterol Ratio (Total-HDL)', 'Ferritin', 'Transferrin',
#  'Iron Binding Capacity, Total', '% Hemoglobin A1c', 'Nucleated Red Cells',
#  'Cholesterol, LDL, Calculated'

# ]
tests = set(tests)
print(len(tests))
tests

15


{'Basophils',
 'Eosinophils',
 'Hematocrit',
 'Hemoglobin 2',
 'Lymphocytes',
 'MCH',
 'MCHC',
 'MCV',
 'Monocytes',
 'Neutrophils',
 'Nucleated Red Cells',
 'Platelet Count',
 'RDW',
 'Red Blood Cells',
 'White Blood Cells'}

In [11]:
tests = ['Hemoglobin']

In [18]:
#Filter by subjects with available waveform data
path = "D:/PulseDB_MIMIC_modified"
labs_filtered = labs[labs.SUBJECT_ID.isin([x[:-4] for x in os.listdir(path)])]
print(labs_filtered.shape, labs.shape)

(2381234, 6) (27788134, 6)


In [19]:
path = 'D:/PulseDB_MIMIC_modified/'
time = '30 min'
interval = pd.Timedelta(time)
time = '30 min new'

for test in tests:
    df_results = pd.DataFrame()
    
    #select the entries for that particular test and add the .mat extension to the subject_id
    files = labs_filtered[(labs_filtered.ITEMID == test)].SUBJECT_ID.value_counts()
    files = files.index + ".mat"

    for file in tqdm(files, desc = f'Processing {test}'):
        subject_id = file[:-4]
        if file in os.listdir(path):
            df = process_file_optimized(path, file, subject_id, interval,
                                        labs_filtered[labs_filtered.ITEMID == test])
            df_results = pd.concat([df_results, df], axis = 0)
    df_results.to_csv(f'D:/simedy/CSVs/Alldata_{test}_{time}.csv', index = False)

Processing Hemoglobin:   0%|          | 0/2340 [00:00<?, ?it/s]

In [7]:
from datetime import datetime, timezone, timedelta

# Define the timezone for GMT+3
gmt_plus_3 = timezone(timedelta(hours=3))

# Get the current time in GMT+3
current_time_gmt_plus_3 = datetime.now(gmt_plus_3)

# Print the current time in GMT+3
print("Current time in GMT+3:", current_time_gmt_plus_3.strftime('%Y-%m-%d %H:%M:%S'))

Current time in GMT+3: 2024-07-23 18:28:41
